<a href="https://colab.research.google.com/github/ImanIra0ui/AI-Search-Shell/blob/master/CVS3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shelve
from nltk.corpus import stopwords
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from sklearn.model_selection import cross_validate
from sklearn import metrics
import numpy as np
from numpy import average
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import sys
from scipy import sparse as sp_sparse
from numpy import mean, std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold, LeavePOut, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import naive_bayes, svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score          # Scikit's libraries for demonstration
from sklearn.model_selection import cross_val_score
from multiscorer.multiscorer  import MultiScorer
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

from google.colab import files
uploaded = files.upload()

from sklearn.metrics import make_scorer, f1_score
scorer = MultiScorer({
    'Accuracy'  : (accuracy_score , {}),
    'Precision' : (precision_score, {'average':'macro'}),
    'Recall'    : (recall_score   , {'average':'macro'}),
    'F1 Score'  : (f1_score, {'average':'macro'})
})

"""nltk.download('wordnet')
nltk.download('omw-1.4')"""


"""Cleaning the data: make text lowercase, remove stopwords and symbols"""

def text_prepare(text):
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile(r'[^\w\s]+')
    STOPWORDS = set(stopwords.words('english'))
    NEWLINE = ('\n')

    text = text.lower()
    text = text.replace(NEWLINE, ' ')
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    text = re.sub(BAD_SYMBOLS_RE, '', text)
    dup = text

    text = ''
    for word in dup.split():
        if word not in STOPWORDS:
            if len(text):
                text = text + " " + word
            else:
                text = word
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(text)

    lemmatizer = nltk.stem.WordNetLemmatizer()
    text = " ".join(lemmatizer.lemmatize(token) for token in tokens)

    return text

def cleanData(data):
    clean = []
    for ex in data:
        clean.append(text_prepare(ex))

    return clean

def cleanData(data):
    clean = []
    for ex in data:
        clean.append(text_prepare(ex))

    return clean


""" Return most popular words in abuse vs non abuse labels"""

def countWords(X_Train, words_counts):
    for sentence in X_Train:
        sentenceS = sentence.split()
        for word in sentenceS:
            words_counts[word] = words_counts.get(word, 0) + 1

def my_bag_of_words(text, words, dict_size):
    result_vector = np.zeros(dict_size)
    for word in text.split():
        if word in words:
            result_vector[words[word]] += 1
    return result_vector

def tfidf_features(X_train):
    """
        X_train, X_val, X_test — samples
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    # Transform the train, test, and val sets and return the result


    tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1, 2), token_pattern='(\S+)')

    X_train = tfidf_vectorizer.fit_transform(X_train)
    # X_val = tfidf_vectorizer.transform(X_val)
    # X_test = tfidf_vectorizer.transform(X_test)
    return X_train, tfidf_vectorizer.vocabulary_

def encode_label (y):
    Encoder = LabelEncoder()
    y = Encoder.fit_transform(y)

    return y

# i = 0
# abuse = []
# nonabuse = []
#
# for item in X_Train:
#     if y_train[i] == "abuse":
#         abuse.append(item)
#     else:
#         nonabuse.append(item)
#     i = i + 1

# finding common unigrams

# countWordsA(abuse)
# most_common_words = sorted(words_counts_a.items(), key=lambda x: x[1], reverse=True)[:30]
# countWordsNA(nonabuse)
# most_common_words = sorted(words_counts_n.items(), key=lambda x: x[1], reverse=True)[:30]
# countWords(X_Train)
# most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:30]


words_counts = {}
DICT_SIZE = sys.maxsize
np.random.seed(500)
with shelve.open("redditAbuseUneven") as d:
    print(list(d.keys()))

    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    labels = d["labelsTrain"]
    labels.extend(d["labelsTest"])
    train = d["XTrain"]   
    train.extend(d["XTest"])
    # X_train = d["data"]
    # y_train = d["labels"]

    # data = d["data"]
    # # labels = d["labels"]
    X_train = train
    y_train = labels
    print(len(X_train))

    X_train = cleanData(X_train)
    # y_train = encode_label (y_train)
    y_train = label_binarize(y_train, classes = ['abuse', 'non_abuse']).flatten()
    print(y_train)

    countWords(X_train, words_counts)
    most_common_words_n = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:30]

    INDEX_TO_WORDS = sorted(words_counts.keys(), key=lambda x: words_counts[x], reverse=True)[:DICT_SIZE]
    WORDS_TO_INDEX = {word: i for i, word in enumerate(INDEX_TO_WORDS)}
    ALL_WORDS = WORDS_TO_INDEX.keys()
    DICT_SIZE = len(WORDS_TO_INDEX)

    X_train_mybag = sp_sparse.vstack(
        [sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])


    X_train_tfidf, tfidf_vocab = tfidf_features(X_train)
    tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}
    
    Naive = naive_bayes.MultinomialNB()

    cv = KFold(n_splits=10, random_state=1, shuffle=True)    
    classifier = LogisticRegression(penalty="elasticnet", C=3, solver="saga", l1_ratio=0.098, max_iter=10000000)
    SV = svm.SVC(decision_function_shape='ovo')
    # Naive = naive_bayes.ComplementNB()

    # cross_val_score(classifier, X_train_tfidf, y_train, scoring=scorer, cv=cv)
    # results = scorer.get_results()
    # print("\nLog Reg + TF-IDF:")
    # for metric_name in results.keys():
    #         average_score = np.average(results[metric_name])
    #         print('%s : %f' % (metric_name, average_score))
    # y_pred = cross_val_predict(classifier, X_train_tfidf, y_train, cv=cv, n_jobs=-1)
    # conf_mat = confusion_matrix(y_train, y_pred)
    # print(conf_mat)

    cross_val_score(SV, X_train_tfidf, y_train, scoring=scorer, cv=cv)  
    results = scorer.get_results()
    print("\nSVM + TF-IDF:")
    for metric_name in results.keys():
            print(metric_name, results[metric_name])
            average_score = np.average(results[metric_name])
            print('%s : %f' % (metric_name, average_score))
    y_pred = cross_val_predict(SV, X_train_tfidf, y_train, cv=cv, n_jobs = -1)
    # conf_mat = confusion_matrix(y_train, y_pred3)
    # print(conf_mat)

    

    # models = [('lr',classifier),('svm',SV)
    # stacking = StackingClassifier(estimators=models)


    # cvs = cross_val_score(classifier, X_train_mybag, y_train, scoring=scorer, cv=cv)
    # results = scorer.get_results()
    # print("\nLog Reg + BOW:")
    # for metric_name in results.keys():
    #         average_score = np.average(results[metric_name])
    #         print('%s : %f' % (metric_name, average_score))
    # y_pred = cross_val_predict(classifier, X_train_mybag, y_train, cv=cv, n_jobs=-1)
    # conf_mat = confusion_matrix(y_train, y_pred)
    # print(conf_mat)

    # cvs2 = cross_val_score(Naive, X_train_mybag, y_train, scoring=scorer, cv=cv)
    # results = scorer.get_results()
    # print("\nNaive Bayes + BOW:")
    # for metric_name in results.keys():
    #         average_score = np.average(results[metric_name])
    #         print('%s : %f' % (metric_name, average_score))
    # y_pred2 = cross_val_predict(Naive, X_train_mybag, y_train, cv=cv, n_jobs=-1)

    data = {'y_Actual':    y_train,
        'y_Predicted': y_pred
        }
    df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
    confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])

    sn.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='.6g')
    plt.show()

    # cvs3 = cross_val_score(SV, X_train_mybag, y_train, scoring=scorer, cv=cv)
    # results = scorer.get_results()
    # print("\nSVM + BOW:")
    # for metric_name in results.keys():
    #         average_score = np.average(results[metric_name])
    #         print('%s : %f' % (metric_name, average_score))
    # y_pred3 = cross_val_predict(SV, X_train_mybag, y_train, cv=cv, n_jobs= 1)
    # conf_mat = confusion_matrix(y_train, y_pred3)
    # print(conf_mat)

    # print("Cross Validation Scores scores for TF-IDF + Log Reg are {}".format(cvs))
    # print('Accuracy: %.3f (%.3f)' % (mean(cvs), std(cvs)))
    # print('\n\n')
    # print (metrics.classification_report(y_train, y_pred))

    # print("Cross Validation Scores scores for TF-IDF + NB are {}".format(cvs2))
    # conf_mat = confusion_matrix(y_train, y_pred2)
    # print(conf_mat)
    # print('Accuracy: %.3f (%.3f)' % (mean(cvs2), std(cvs2)))
    # print('\n\n')

    # print("Cross Validation Scores scores for TF-IDF + SVM are {}".format(cvs3))
    # conf_mat = confusion_matrix(y_train, y_pred)
    # print(conf_mat)
    # print('Accuracy: %.3f (%.3f)' % (mean(cvs3), std(cvs3)))
    # print('\n\n')

    # cvs = cross_val_score(classifier, X_train_mybag, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    # cvs2 = cross_val_score(Naive, X_train_mybag, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    # cvs3 = cross_val_score(SV, X_train_mybag, y_train, scoring='accuracy', cv=cv, n_jobs= 1)

    # y_pred = cross_val_predict(classifier, X_train_mybag, y_train, cv=cv, n_jobs=-1)
    # y_pred2 = cross_val_predict(Naive, X_train_mybag, y_train, cv=cv, n_jobs=-1)
    # y_pred3 = cross_val_predict(SV, X_train_mybag, y_train, cv=cv, n_jobs= 1)

    # print("Cross Validation Scores for BOW + Log Reg are {}".format(cvs))
    # conf_mat = confusion_matrix(y_train, y_pred)
    # print(conf_mat)
    # print('Accuracy: %.3f (%.3f)' % (mean(cvs), std(cvs)))
    # print('\n\n')

    # print("Cross Validation Scores scores for BOW + NB are {}".format(cvs2))
    # conf_mat = confusion_matrix(y_train, y_pred2)
    # print(conf_mat)
    # print('Accuracy: %.3f (%.3f)' % (mean(cvs2), std(cvs2)))
    # print('\n\n')

    # print("Cross Validation Scores for BOW + SVM are {}".format(cvs3))
    # conf_mat = confusion_matrix(y_train, y_pred3)
    # print(conf_mat)
    # print('Accuracy: %.3f (%.3f)' % (mean(cvs3), std(cvs3)))
    # print('\n\n')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Saving redditAbuseUneven.bak to redditAbuseUneven.bak
Saving redditAbuseUneven.dat to redditAbuseUneven.dat
Saving redditAbuseUneven.dir to redditAbuseUneven.dir
['subIdsTest', 'labelsTest', 'XTrain', 'XTest', 'labelsTrain', 'subIdsTrain']
18356
[1 1 0 ... 1 1 1]

SVM + TF-IDF:
Accuracy [0.9629629629629629, 0.9684095860566448, 0.9580610021786492, 0.9624183006535948, 0.9705882352941176, 0.9607843137254902, 0.9607629427792915, 0.9662125340599454, 0.95858310626703, 0.9629427792915531]
Accuracy : 0.963173
Precision [0.9412699145693835, 0.9572879354151935, 0.9713395890632583, 0.9601763555839284, 0.9452909502016333, 0.9450305500381446, 0.9796725014116319, 0.9823963657013061, 0.9656439393939394, 0.9561461115245975]
Precision : 0.960425
Recall [0.7338380896913221, 0.7793202625546396, 0.7376353514549644, 0.7528500271848747, 0.7895611752520021, 0.7674587668680994, 0.7352941176470589, 0.7720588235294117, 0.7477069026662797, 0.7669373916459566]
Recall : 0.758266
F1 Score [0.8013745704467354, 0.843

In [ ]:
from pynput.mouse import Controller,Button
import time
mouse = Controller()

while True:
    mouse.click(Button.left,1)
    print('clicked')

    time.sleep(5)

In [2]:
!git clone https://github.com/StKyr/multiscorer/

Cloning into 'multiscorer'...
remote: Enumerating objects: 49, done.
remote: Total 49 (delta 0), reused 0 (delta 0), pack-reused 49
Unpacking objects: 100% (49/49), done.


In [ ]:
print("Cross Validation Scores for BOW + Log Reg are {}".format(cvs))
print('Accuracy: %.3f (%.3f)' % (mean(cvs), std(cvs)))

Cross Validation Scores for BOW + Log Reg are [0.97565663 0.97117233 0.97051282 0.97179487 0.96666667 0.96987179
 0.97564103 0.97115385 0.97371795 0.96217949]
Accuracy: 0.971 (0.004)
